<h2 style="text-align: center;" markdown="1">K-nearesr neighbour Model</h2>

In [1]:
import pandas as pd

### First We will use the 2013 data as Train and Test Set

In [5]:
train = pd.read_csv('/home/smita/expedia/setting2/train_S2_A1.csv')
test = pd.read_csv('/home/smita/expedia/setting2/test_S2_A1.csv')

In [6]:
train.shape[0] 

4494597

In [7]:
test.shape[0]

612239

In [10]:
train = train.iloc[:,1:]
test = test.iloc[:, 1:]

In [11]:
train.head(1)

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,month,year,day,time
0,2,3,66,462,41898,2716.6746,1482,0,0,1,...,0,1,2,50,214,28,2,2013,15,13:18:43


In [12]:
test.head(1)

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,month,year,day,time
0,30,4,195,548,56440,NaN,1048,0,1,9,...,1,1,0,185,185,58,6,2013,15,15:38:05


<h2 style="text-align: center;" markdown="1">Feature Engineering</h2>

#### Function to Create popularity score based on the click/booking for pair of (user_city, srch_destination)

In [17]:
from collections import defaultdict
def createCityDestScoreDict(df):
    to_from_dict = {}
    columns = df.columns.values
    for index, row in df.iterrows():
            user_city = str(row['user_location_city'])
            srch_dest = str(row['srch_destination_id'])
            is_booking = row['is_booking']
            hotel_cluster = int(row['hotel_cluster'])
              
            if is_booking == '1':
                is_booking = 1 * 0.75
            else:
                is_booking = 1 * 0.25
    
            if user_city and srch_dest:   
                if (user_city,srch_dest) in to_from_dict:
                    if not to_from_dict[(user_city,srch_dest)].get(hotel_cluster):
                        to_from_dict[(user_city,srch_dest)][hotel_cluster] = is_booking
                    else:
                         to_from_dict[(user_city,srch_dest)][hotel_cluster] +=is_booking
                else:
                     to_from_dict[(user_city, srch_dest)] = {hotel_cluster: is_booking}
    return to_from_dict   

#### Fuction to get first n (key:values) from dictionary

In [18]:
from itertools import islice
def sliceDict(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

#### Create popularity_score and add to train set

In [19]:
def createPopScore(df, to_from_dict):
    for index, row in df.iterrows():
        userCity = str(row['user_location_city'])
        srchCity = str(row['srch_destination_id'])
        hotel_cluster = int(row['hotel_cluster']) 
  
        if (userCity,srchCity) in to_from_dict:
                pop_score = to_from_dict[(userCity, srchCity)].get(hotel_cluster, None)
        else:
                pop_score = 0
        df.set_value(index,'pop_score', pop_score)
    return df   

#### Function Duration_of_Stay

In [20]:
from datetime import datetime
def durationOfStay(df):
    df["srch_ci"] = pd.to_datetime(df["srch_ci"], format='%Y-%m-%d', errors="coerce")
    df["srch_co"] = pd.to_datetime(df["srch_co"], format='%Y-%m-%d', errors="coerce")
    df["time_of_stay"] = (df["srch_co"]-df["srch_ci"]).astype('timedelta64[h]')
    df["time_of_stay"] = df["time_of_stay"]/24
    return df

#### Get the (user-city, source: score) dictionry and print 1st five

In [21]:
to_from_dict = createCityDestScoreDict(train)
items = sliceDict(5, to_from_dict.iteritems())
items

[(('49966', '8259'),
  {36: 0.25, 42: 0.25, 46: 0.5, 58: 0.5, 69: 0.25, 90: 0.25, 97: 1.5}),
 (('25538', '12243'), {5: 0.25, 37: 0.5, 53: 0.25, 55: 0.25}),
 (('3169', '25506'), {72: 0.25}),
 (('7317', '1455'), {28: 0.25}),
 (('876', '4330'), {32: 0.25})]

### 1. Add pop_score in Train & Test

In [22]:
train = createPopScore(train, to_from_dict=to_from_dict)

In [23]:
train.iloc[0:1, 15:]

,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,month,year,day,time,pop_score
0,8857,1,0,1,2,50,214,28,2,2013,15,13:18:43,0.5


In [24]:
test = createPopScore(test, to_from_dict=to_from_dict)

In [25]:
test.iloc[0:1, 15:]

,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,month,year,day,time,pop_score
0,1385,1,1,1,0,185,185,58,6,2013,15,15:38:05,4.0


### 2.  Add duration_of_stay in Train

In [26]:
train = durationOfStay(train)
test = durationOfStay(test)

In [27]:
train.iloc[0:1, 15:]

,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,month,year,day,time,pop_score,time_of_stay
0,8857,1,0,1,2,50,214,28,2,2013,15,13:18:43,0.5,5.0


In [29]:
test.iloc[0:1, 15:]

,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,month,year,day,time,pop_score,time_of_stay
0,1385,1,1,1,0,185,185,58,6,2013,15,15:38:05,4.0,8.0


In [39]:
train.to_csv('/home/smita/expedia/setting2/knn/knnTrain13.csv', index_label= False)

In [40]:
test.to_csv('/home/smita/expedia/setting2/knn/knnTest13.csv', index_label= False)

In [ ]:
### KNN files : knnTrain13, knnTest13